# A to Z fitting HI model according to Hypo 2 (combined model)

In [1]:
from hypo2.api import RunTime, FitTime
from hypo2.config import RunConfig
import os
from hypo2.dataset import Dataset
from hypo2.model import HIModel
import hypo2.api as api
import random
import numpy as np
from hypo2.preprocessing import WordSegmentator
from hypo2.normalization import Normalizator
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [2]:
import torch.nn as nn
import torch
from torchvision.models.inception import Inception3

In [3]:
config = RunConfig()

In [4]:
paths = FitTime.gen_paths("C:/ml_prj/SchoolHWA/data2")
config.CLASS_COUNT = 2
ws = WordSegmentator(config)
nm = Normalizator(config)

In [ ]:
ds = Dataset(config, ws, nm)

In [ ]:
#X, y = ds.gen_dataset([paths[0] + paths[1], paths[2]], True)
X, y = ds.gen_dataset([paths[-1], paths[-3]], True)

50.0 % 57.14 %


In [ ]:
Xl = list(X)
yl = list(y)
i = 0
while i < len(Xl):
    if Xl[i].mean() < -0.46:
        del Xl[i]
        del yl[i]
    else:
        i += 1
X = np.stack(Xl)
y = np.stack(yl)

In [ ]:
def add_padding(res, pattern):
    before_0 = -(res.shape[1] - pattern[0]) // 2
    after_0 = -(res.shape[1] - pattern[0]) // 2 + (res.shape[1] - pattern[0]) % 2
    before_1 = -(res.shape[0] - pattern[1]) // 2
    after_1 = -(res.shape[0] - pattern[1]) // 2 + (res.shape[0] - pattern[1]) % 2
    return np.pad(res, [(before_1, after_1), (before_0, after_0), (0, 0)], mode="constant", constant_values=1)

def add_padding_4(res, pattern):
    before_0 = -(res.shape[3] - pattern[0]) // 2
    after_0 = -(res.shape[3] - pattern[0]) // 2 + (res.shape[3] - pattern[0]) % 2
    before_1 = -(res.shape[2] - pattern[1]) // 2
    after_1 = -(res.shape[2] - pattern[1]) // 2 + (res.shape[2] - pattern[1]) % 2
    return np.pad(res, [(0, 0), (0, 0), (before_1, after_1), (before_0, after_0)], mode="constant", constant_values=1)

In [ ]:
FF = int((1 - y.mean()) * len(X))
dX1 = list(X[:FF])
dX2 = list(X[FF:])
dXtest1 = []
dXtest2 = []
for i in range((len(dX1) + len(dX2)) // 7):
    f1 = random.randint(0, len(dX1) - 1)
    f2 = random.randint(0, len(dX2) - 1)
    dXtest1.append(dX1[f1])
    dXtest2.append(dX2[f2])
    del dX1[f1]
    del dX2[f2]
dX1 = np.stack(dX1)
dX2 = np.stack(dX2)
dX = [dX1, dX2]
dXtest1 = np.stack(dXtest1)
dXtest2 = np.stack(dXtest2)
dXtest = [dXtest1, dXtest2]

In [ ]:
dX1.shape, dX2.shape, dXtest1.shape, dXtest2.shape

In [ ]:
DEVICE = torch.device("cuda")

In [ ]:
def resort_and_noise(dX, dX1, dX2, samples):
    rX1 = []
    rX2 = []
    ry = []
    for i in range(samples):
        if i % 1000 == 999:
            print(i)
        if random.random() > 0.5:
            source = random.choice(dX)
            t1 = random.choice(source)
            t2 = random.choice(source)
            ry.append(1.0)
        else:
            t1 = random.choice(dX1)
            t2 = random.choice(dX2)
            ry.append(0.0)
        #t1noise = np.stack([np.random.randn(299, 299)] * 3, axis=2) / 30
        #t2noise = np.stack([np.random.randn(299, 299)] * 3, axis=2) / 30
        #t1 += t1noise
        #t2 += t2noise
        rX1.append(t1)
        rX2.append(t2)

    fX1 = torch.from_numpy(np.stack(rX1)).transpose(1, 3).transpose(2, 3).type(torch.float)
    fX2 = torch.from_numpy(np.stack(rX2)).transpose(1, 3).transpose(2, 3).type(torch.float)
    fy = torch.tensor(ry)
    return fX1, fX2, fy

fX1, fX2, fy = resort_and_noise(dX, dX1, dX2, 45000)
fXtest1, fXtest2, ftesty = resort_and_noise(dXtest, dXtest1, dXtest2, 2000)

In [ ]:
print("  Первый человек" + " " * 45 + "Второй человек")
fig = plt.figure(figsize=[16, 16])
for i in range(36):
    plt.subplot(18,2,i+1)
    if i % 2 == 1:
        plt.imshow(dX2[i] + 1/2)
    else:
        plt.imshow(dX1[i] + 1/2)
    fig.axes[i].get_xaxis().set_visible(False)
    fig.axes[i].get_yaxis().set_visible(False)

In [ ]:
for i in range(10):
    plt.figure(figsize=[16, 2])
    print("Один и тот же человек:" if fy[i] == 0 else "Разные люди:")
    plt.subplot(1, 2, 1)
    plt.imshow(fX1[i].transpose(0, 2).transpose(0, 1) + 0.5)
    plt.subplot(1, 2, 2)
    plt.imshow(fX2[i].transpose(0, 2).transpose(0, 1) + 0.5)
    plt.show()

In [ ]:
print("Consumed memory:", round(2 * fX1.shape[0] * fX1.shape[1] * fX1.shape[2] * fX1.shape[3] * 4 / 1000000000, 1), "Gb")

In [ ]:
class Norm(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x / 100

In [ ]:
lllll = []

In [ ]:
'''
            nn.Conv2d(3, 3, kernel_size=[3, 3], padding=1),
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 3, kernel_size=[3, 3], padding=1),
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 3, kernel_size=[3, 3], padding=1),
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 3, kernel_size=[3, 3], padding=1),
            nn.BatchNorm2d(3),
            '''
None

In [ ]:
def mp__(r):
    s = 1
    for i in r:
        s *= i
    return s

class Sigm(nn.Module):
    def forward(self, x):
        return 1 / (1 + 1.4 ** (-(x - 5)))

class Flatten(nn.Module):
    def forward(self, x):
        return x.reshape(x.shape[0], (mp__(x.shape[1:])))

class MultiKernelLayer(nn.Module):
    def __init__(self, kernels, dev, ch=3):
        super().__init__()
        self.lenk = len(kernels)
        for i in range(self.lenk):
            exec("self.module" + str(i) + " = nn.Conv2d(ch, ch, kernel_size=kernels[i], padding=kernels[i]//2).to(dev)")
        self.final = nn.BatchNorm2d(ch)
    
    def forward(self, x):
        r = 0
        for i in range(self.lenk):
            r += eval("self.module" + str(i) + "(x)")
        return self.final(r)
    
class DyPa(nn.Module):
    def __init__(self, layer_count, kernels, size, class_count, dev):
        super().__init__()
        for i in range(layer_count):
            exec("self.layer" + str(i) + " = MultiKernelLayer(kernels, dev)")
        self.layer_count = layer_count
        self.final = nn.Sequential(
            Flatten(),
            nn.Linear(size[0] * size[1] * 3, class_count),
            Sigm()
        )
        
    def forward(self, x):
        r = 0
        x1 = x
        for layer in range(self.layer_count):
            f = eval("self.layer" + str(layer) + "(x1)")
            r += f
            x1 = f
        return self.final(r)
        
    
class FixedInc3(nn.Module):
    def __init__(self, CLASS_COUNT, dev):
        super().__init__()
        self.t = Inception3(CLASS_COUNT)
        '''
        self.t = nn.Sequential(
            nn.Conv2d(3, 3, kernel_size=[3, 3], padding=1),
            nn.BatchNorm2d(3),
            Flatten(),
            nn.Linear(299 * 299 * 3, CLASS_COUNT),
            Sigm()
        )
        '''
        #self.t = DyPa(15, [3, 5, 7, 9, 11, 13, 15], (299, 299), CLASS_COUNT, dev)
    
    def forward(self, x):
        return self.t(x)[0]
    
class Siames(nn.Module):
    def __init__(self, cfg, device):
        super().__init__()
        self.tw = FixedInc3(cfg.FEATURES_COUNT, device)
        self.cfg = cfg
        
        self.comparator = nn.Sequential(
            #nn.Linear(cfg.FEATURES_COUNT, cfg.FEATURES_COUNT),
            #Norm(),
            nn.Linear(cfg.FEATURES_COUNT, cfg.FEATURES_COUNT),
            Norm(),
            nn.Linear(cfg.FEATURES_COUNT, cfg.FEATURES_COUNT),
            Norm(),
            nn.Sigmoid(),
            nn.Linear(cfg.FEATURES_COUNT, 1),
            #Norm(),
            #nn.Sigmoid()
            Sigm()
        )
        
        self.mp = (torch.zeros((300, 1)) + 1).type(torch.float).to(device)
        self.sig = Sigm()
    def forward(self, x1, x2):
        f1 = self.tw(x1)
        #print("1:", f1[0][0])
        f2 = self.tw(x2)
        #print("2:", f2[0][0])
        f3 = (f1 - f2) ** 2 * 0.003
        #print("3:", f3[0][0])
        #print(f3.mean())
        #lllll.append(f3)
        #return self.comparator(f3)
        f4 = torch.mm(f3, self.mp)
        #print("4:", f4[0][0])
        return self.sig(f4)

In [ ]:
#model = Siames(config, torch.device("cuda")).to(DEVICE)
model = torch.load("D:/main/ml_prj/SchoolHWA/siames_models/hi1559451128.h5")

In [ ]:
model(torch.randn((5, 3, 299, 299)).to(DEVICE), torch.randn((5, 3, 299, 299)).to(DEVICE))

In [ ]:
BS = 5

In [ ]:
def runtime_preprocess(bX, dev):
    bX = torch.tensor(add_padding_4(1 - (bX + 1/2), (299, 299)))
    tnoise = torch.from_numpy(np.stack([np.stack([np.random.randn(299, 299)] * 3, axis=2) / 30 for j in range(bX.shape[0])])).transpose(1, 3)
    return 1 - (bX + tnoise.type(torch.float)).to(dev)

In [ ]:
def tttttttttttttttttest():
    batchid = random.randint(0, len(fX1) - BS - 1)
    bX1 = runtime_preprocess(fX1[batchid:batchid+BS], DEVICE)
    bX2 = runtime_preprocess(fX2[batchid:batchid+BS], DEVICE)
    ytrue = fy[batchid:batchid+BS].to(DEVICE)
    ypred = model(bX1, bX2)
    return ypred

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1.0e-3)

In [ ]:
def mfloss(pred, true):
    return ((pred - true) ** 2 * (torch.abs(pred - 0.5) + 0.1)).mean()
#crit = mfloss
crit = nn.BCELoss()
#crit = nn.MSELoss()

In [ ]:
from IPython.display import clear_output
DEVICE = torch.device("cuda")

losses = []
all = 0
s = 0
accs = []

In [ ]:
for i in range(0):
    batchid = random.randint(0, len(fX1) - BS - 1)
    bX1 = runtime_preprocess(fX1[batchid:batchid+BS], DEVICE)
    bX2 = runtime_preprocess(fX2[batchid:batchid+BS], DEVICE)
    ytrue = fy[batchid:batchid+BS].to(DEVICE)
    ypred = model(bX1, bX2)
    loss = crit(ypred, ytrue)
    loss.backward()
    opt.step()
    opt.zero_grad()
    losses.append(loss.item())
    clear_output(True)
    
    _all = BS
    _s = (BS - ((ypred.reshape(BS) > 0.5).type(torch.float) - (ytrue.reshape(BS) > 0.5).type(torch.float)).abs().sum()).item()
    all += _all
    s += _s
    acc = _s / _all
    accs.append(acc)
    
    plt.plot(losses, label="loss")
    #plt.plot(accs, label="acc")
    plt.legend()
    plt.show()
    
    print("Loss:", loss.item())

In [ ]:
import time
#torch.save(model, "D:/main/ml_prj/SchoolHWA/siames_models/hi" + str(int(time.time())) + ".h5")

In [ ]:
#print("Train accuracy:", sum(accs[-90:]) / len(accs[-90:]))

In [ ]:
all = 0
s = 0

In [ ]:
for i in range(300):
    batchid = random.randint(0, len(fXtest1) - BS - 1)
    bX1 = runtime_preprocess(fXtest1[batchid:batchid+BS], DEVICE)
    bX2 = runtime_preprocess(fXtest2[batchid:batchid+BS], DEVICE)
    ytrue = ftesty[batchid:batchid+BS].to(DEVICE)
    ypred = model(bX1, bX2)
    all += BS
    s += (BS - ((ypred.reshape(BS) > 0.5).type(torch.float) - (ytrue.reshape(BS) > 0.5).type(torch.float)).abs().sum()).item()

In [ ]:
print("Final accuracy:", s / all)